<a href="https://colab.research.google.com/github/TIZadid/ML_Toolkit/blob/main/My_Machine_Learning_notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Packages

In [ ]:
import pandas as pd
import numpy as np
import matplotlib as plt

# Dataset


## Sklearn Datasets

In [ ]:
from sklearn import datasets

In [ ]:
dataset = datasets.load_breast_cancer()

In [ ]:
df = pd.DataFrame(dataset.data, columns=dataset.feature_names)
df['target'] = dataset.target
X = df.drop(columns=['target'])
y = df['target']

## csv extracted dataset

## **Train Test Splitting**

In [ ]:
from sklearn.model_selection import train_test_split
splitRatio=0.2
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size =splitRatio,
                                                    stratify=None)
#stratify used to equally sample target class

# ML Models

### Decision Tree Classifier/Regressor

In [ ]:
from sklearn.tree import DecisionTreeClassifier

In [ ]:
dt=DecisionTreeClassifier()
dt.fit(X_train,y_train)
y_pred=dt.predict(X_test)
y_pred_proba=dt.predict_proba(X_test)[:,1]  #for positive class
myModel=dt

### K Nearest Neighbours

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
knn = KNeighborsClassifier(n_neighbors=6)
knn.fit(X_train, y_train)
y_pred=knn.predict(X_test)
print(knn.score(X_test, y_test))
myModel=knn

0.9122807017543859


# XGBOOST

Used(Best) for supervised learning where,


*   Number of feature < number of sample
*   Mixture of numerical and categorical variable or only numerical variables

Not used in Compuer vision, NLP and small datasets 


In [ ]:
import xgboost as xgb

In [ ]:
xg_cl = xgb.XGBClassifier(objective='binary:logistic', n_estimators=10, seed=123)
xg_cl.fit(X_train,y_train)
y_pred = xg_cl.predict(X_test)
accuracy = float(np.sum(y_pred==y_test))/y_test.shape[0]
print("accuracy: %f" % (accuracy))

accuracy: 0.929825


## XGBOOST with cross validation

`nfold` is the number of cross-validation folds, `num_boost_round` is the number of trees we want to build

In [ ]:
myDMatrix = xgb.DMatrix(data=X, label=y)
params = {"objective":"reg:logistic", "max_depth":3}
cv_results = xgb.cv(dtrain=myDMatrix, params=params, 
                  nfold=3, num_boost_round=5, 
                  metrics="error", as_pandas=True, seed=123)

In [ ]:
print(cv_results)
print(((1-cv_results["test-error-mean"]).iloc[-1]))

   train-error-mean  train-error-std  test-error-mean  test-error-std
0          0.025480         0.002451         0.066824        0.019564
1          0.021969         0.001257         0.061524        0.013876
2          0.014945         0.006589         0.056252        0.010004
3          0.012306         0.003300         0.052734        0.011418
4          0.010549         0.004314         0.054497        0.012485
0.9455026666666666


# Scores

### For Classification Problems

In [ ]:
from sklearn.metrics import accuracy_score
acc_score=accuracy_score(y_pred,y_test)
print("Accuracy score: ",acc_score)

Accuracy score:  0.9298245614035088


### For regression Problems

In [ ]:
from sklearn.metrics import mean_squared_error as MSE
mse=MSE(y_pred,y_test)
rmse=mse**.5
print("Root mean squared error score: ",rmse)

Root mean squared error score:  0.26490647141300877


### For PredicProba

In [ ]:
from sklearn.metrics import roc_auc_score
roc_auc=roc_auc_score(y_test,y_pred_proba)
print("ROC score: ",roc_auc)

ROC score:  0.928414442700157


# Ensemble Learning

# HyperParameter Tuning

---


1.   Grid Search
2.   







### Grid Search

In [ ]:
from sklearn.model_selection import GridSearchCV
params_dt={
    'max_depth': [3,4,5,6],
    'min_samples_leaf': [.04,.06,.08],
    'max_features': [.2,.4,.6,.8]
}
myGrid=GridSearchCV(estimator=dt,
                     param_grid=params_dt,
                     scoring='accuracy',  
                     #scoring='neg_mean_squared_error',
                     cv=10,
                     n_jobs=-1)
myGrid.fit(X_train,y_train)

GridSearchCV(cv=10, error_score=nan,
             estimator=DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features=None,
                                              max_leaf_nodes=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              presort='deprecated',
                                              random_state=None,
                                              splitter='best'),
             iid='deprecated', n_jobs=-1,
             param_grid={'max_depth': [3, 4, 5, 6],
                         'm

In [ ]:
best_hyperparams=myGrid.best_params_
print('Best Hyperparameters: ', best_hyperparams)

Best Hyperparameters:  {'max_depth': 5, 'max_features': 0.8, 'min_samples_leaf': 0.04}


In [ ]:
best_model=myGrid.best_estimator_